# GENERAMOS LAS INSTANCIAS:

##### Importamos las librerias a usar

In [1]:
import math, subprocess, time, random, os, shutil, sys
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
def save_instance(dataset, instance_name, b,c, m):
    with open(F"instances/{dataset}/{instance_name}.txt", "w") as f:
        print(len(c), m, file=f)
        for j in range(0,len(c)): 
            print(b[j], file=f, end=" ")
            print(c[j], file=f, end=" \n")


In [3]:
def clean_data(nombre):
    try:
        shutil.rmtree("instances/"+nombre)
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))
    if os.path.isfile("results/result.csv"):
        df = pd.read_csv("results/result.csv")
        df =  df[df.dataset != nombre]
        df.to_csv("results/result.csv", index=False)
        

    os.makedirs("instances/"+nombre, exist_ok=False)

#### En esta seccion nos centraremos en correr los experimentos con las instancias generadass mas arriba

#### Funciones a usar:

In [61]:
def run_program(method, test_file):
    #corre el programa con el metodo indicado en el test file indicado y devuelve el tiempo tomado en ejecutar
    #leemos la instancia
    
    start = time.time() #comenzamos a medir el timepo
    proc = subprocess.Popen(["../src/tp1",test_file,method],stdout = subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True) #lanzamos proceso
    #stdout = subprocess.PIPE
    
    exit_code = proc.wait()#esperamos a que el proceso termien
    
    end = time.time() #terminamos de medir el tiempo
    
    #mandamos error de ser necesario 
    if exit_code != 0: raise RuntimeError(F"Hubo un error en la experimentacion para el algoritmo: {method} con la instancia {test_file}.")
    
    return end-start #devolvemos el timepo
    

In [6]:
def run_experiments(name = "exp"):
    #recibe un nombre para darle al archivo de salida y lo que hace es correr todos los experimentos que se encuentren en la lista llamada experiments que debe ser generada con anterioridad
    columns = ["dataset","exp_name","n", "m", "method", "time"]
    rows = []
    j = 1
    T =10 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
    explen = str(len(experiments))
    
    
    for exp in tqdm(experiments, "Completed experiments"):
#         print(exp)
        times = []
        for i in range(0, T):
            times.append(run_program(exp[3], exp[4]))
        rows.append([exp[0], name, exp[1], exp[2],exp[3], np.mean(times)])
        j +=1
    result = pd.DataFrame(rows, columns=columns)
    
    if os.path.isfile("results/result.csv"):
        df = pd.read_csv("results/result.csv")
        df = pd.concat([df, result], ignore_index = True, axis = 0)
    else:
        df = result
    
    
    df.to_csv("results/result.csv", index=False, header=True)

## Corrida de experimentos

#### Leemos los indices de las instancias generadas anteriormente 

## Experimento 1

In [101]:
def exp1(k=41):
    
    filas_indice = []
    nombre = "peor_btf"
    if (k!=41): nombre = "peor_btf_bf"


#     clean_data(nombre)
    
    
    for n in tqdm(range(10, k,1), "Generating instances"):
        
        #damos valores a n,w,b,c
        m = n+2
        c = [1 for i in range(0,n)]
        b = [(n-i) for i in range(0,n)]
        b[-1] = 1000
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [103]:
clean_data("peor_btf")

In [104]:
exp1()

Generating instances:   0%|          | 0/31 [00:00<?, ?it/s]

In [105]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "peor_btf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/124 [00:00<?, ?it/s]

## Experimento 2

In [177]:
def exp2(k=41): #este experimento sera utilizado para la hipotesis 1

    filas_indice = []
    nombre = "peor_bto"
    if (k!=41): nombre = "peor_bto_bf"


#     clean_data(nombre)
    
    
    for n in tqdm(range(10, k,1), "Generating instances"):
        
        #damos valores a n,w,b,c
        m = 5
        c = [1 for i in range(0,n)]
        b = [0 for i in range(0,n)]
        b[-1] = n**2
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [ ]:
clean_data("peor_bto")

In [ ]:
exp2()

In [25]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "peor_bto"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/124 [00:00<?, ?it/s]

## Experimento 3

In [45]:
def exp3(): #este experimento sera usado para la hipotesis 2
    
    for i in tqdm(range(1,6), "Generating instances"):
        
        filas_indice = []

        nombre = "m_"+str(i)

    #     clean_data(nombre)
        for n in range(2, 10001,500):


            #damos valores a n,w,b,c
            m= (i*1000)
            b = [1 for j in range(0,n)]
            c = [1 for j in range(0,n)]

            save_instance(nombre, nombre+F"-{n}",b,c, m)
            filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
        pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

    #######################
    


In [47]:
for i in range(1,6):
    clean_data("m_"+str(i))


In [48]:
exp3()

Generating instances:   0%|          | 0/5 [00:00<?, ?it/s]

In [52]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

for i in tqdm(range(1,6), "Preparing experiments:"):
    instance = "m_"+str(i)
    df_data = pd.read_csv("instances/"+instance+"/indice.csv")
    for k in range(0, len(df_data)):
        row_k = df_data.iloc[k]
        
        experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

    
#corremos los experimentos
run_experiments("m_exp")


Preparing experiments::   0%|          | 0/5 [00:00<?, ?it/s]

Completed experiments:   0%|          | 0/100 [00:00<?, ?it/s]

## Experimento 4

In [72]:
def exp4(): 
    
    for i in tqdm(range(1,6), "Generating instances"):
        
        filas_indice = []
    
        nombre = "n_"+str(i)

    #     clean_data(nombre)
        n = 10 + (i*10)
        for m in range(2, 100001,1000):


            #damos valores a n,w,b,c

            b = [1 for j in range(0,n)]
            c = [1 for j in range(0,n)]

            save_instance(nombre, nombre+F"-{m}",b,c, m)
            filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{m}.txt"])
        pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

    #######################
    
    

In [73]:
for i in range(1,6):
    clean_data("n_"+str(i))


In [74]:
exp4()

Generating instances:   0%|          | 0/5 [00:00<?, ?it/s]

In [75]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

for i in tqdm(range(1,6), "Preparing experiments:"):
    instance = "n_"+str(i)
    df_data = pd.read_csv("instances/"+instance+"/indice.csv")
    for k in range(0, len(df_data)):
        row_k = df_data.iloc[k]
        experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])
    
#corremos los experimentos
run_experiments("n_exp")


Preparing experiments::   0%|          | 0/5 [00:00<?, ?it/s]

Completed experiments:   0%|          | 0/500 [00:00<?, ?it/s]

# EXPERIMENTO DP PEOR QUE BT

In [57]:
def exp7(k=17):
    
    filas_indice = []
    nombre = "peor_dp"


#     clean_data(nombre)
    #matriz se banca 2^24
    
    for n in tqdm(range(15,25,1), "Generating instances"):
        
        #damos valores a n,w,b,c
        m = int((2**n)/(n))
        c = [1 for i in range(0,n)]
        b = [(n-i) for i in range(0,n)]
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [58]:
clean_data("peor_dp")

In [59]:
exp7()

Generating instances:   0%|          | 0/10 [00:00<?, ?it/s]

In [62]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "peor_dp"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

run_experiments("peor_dp_exp")


Completed experiments:   0%|          | 0/40 [00:00<?, ?it/s]

## Experimento particular bt

In [170]:
def exp5(): 
    filas_indice = []
    
    nombre = "particular_bt"
        
#     clean_data(nombre)
    for n in tqdm(range(10, 41,1), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m = int(n/4)
        c = [1 for i in range(0,n)]
        b = [i**2 for i in range(0,n)]
        
#         m= np.random.randint(100,1000)
#         b = np.random.randint(100, size=n)
#         c = np.random.randint(100, size=n)
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)



In [171]:
clean_data("particular_bt")

In [172]:
exp5()

Generating instances:   0%|          | 0/31 [00:00<?, ?it/s]

In [173]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "particular_bt"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/93 [00:00<?, ?it/s]

## EXPERIMENTO FB

Realizamos la experimentacion con fuerza bruta aparte debido al tiempo de computo requerido para realizarla

In [183]:
def exp6(): 
    filas_indice = []
    
    nombre = "random_bf"
        
#     clean_data(nombre)
    for n in tqdm(range(10, 31,1), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m = np.random.randint(0, 1000)
        c = [np.random.randint(0, 100) for i in range(0,n)]
        b = [np.random.randint(0, 100) for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)



In [184]:
clean_data("peor_bto_bf")
clean_data("peor_btf_bf")
clean_data("random_bf")

In [187]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos


instance = "peor_bto_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])


instance = "peor_btf_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])

instance = "random_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    
#corremos los experimentos
run_experiments()


Completed experiments:   0%|          | 0/63 [00:00<?, ?it/s]